## Import

Notebook de ce que pourrait être le workflow d'import des datasets.

In [1]:
import pandas as pd

pd.options.display.max_columns = 50


## Dataset initial

In [2]:

# dataframe initial
my_dataset =  pd.DataFrame({
                'cnit': pd.Series(dtype='object'),
                'mine': pd.Series(dtype='object'),
                'marque': pd.Series(dtype='object'),
                'modele': pd.Series(dtype='object'),
                'dscom': pd.Series(dtype='object'),
                'carburant': pd.Series(dtype='object'),
                'hybride': pd.Series(dtype='object'),
                'puiss_admin': pd.Series(dtype='float'),
                'puiss_max': pd.Series(dtype='float'),
                'boite_v': pd.Series(dtype='object'),
                'conso_urb': pd.Series(dtype='float'),
                'conso_exurb': pd.Series(dtype='float'),
                'conso_mixte': pd.Series(dtype='float'),
                'masse_v_min': pd.Series(dtype='int'),
                'masse_v_max': pd.Series(dtype='int'),    
                'co2': pd.Series(dtype='float'),
                'co_typ_1': pd.Series(dtype='float'),
                'hc': pd.Series(dtype='float'),
                'nox': pd.Series(dtype='float'),
                'hcnox': pd.Series(dtype='float'),
                'particules': pd.Series(dtype='float')                  
                })

## Import des datasets

In [3]:

df_2011 = pd.read_excel("datasets/2011_BASE CL MAJ JUIN 2011.xls")
df_2012 = pd.read_csv("datasets/2012_BASE CL MAJ JUIN 2012.csv", sep=';',encoding = 'latin-1')


## Normalisation des colonnes

In [4]:
df_2011_cols = {"cnit" : "cnit",
                "tvv" : "mine",
                "lib_mrq" : "marque",
                "lib_mod" : "modele",
                "dscom" : "dscom",
                "typ_cbr" : "carburant",
                "puiss_admin_98" : "puiss_admin",
                "puiss_max" : "puiss_max",
                "typ_boite_nb_rapp" : "boite_v",
                "conso_urb" : "conso_urb",
                "conso_exurb" : "conso_exurb",
                "conso_mixte" : "conso_mixte",
                "co2" : "co2"
               }

to_drop = []
for c in df_2011.columns:
    if c not in df_2011_cols.keys():
        to_drop.append(c)

df_2011 = df_2011.drop(columns=to_drop).rename(columns=df_2011_cols)

In [5]:
df_2012_cols = {"cnit" : "cnit",
                "tvv" : "mine",
                "lib_mrq" : "marque",
                "lib_mod" : "modele",
                "dscom" : "dscom",
                "typ_cbr" : "carburant",
                "hybride" : "hybride",
                "puiss_admin_98" : "puiss_admin",
                "puiss_max" : "puiss_max",
                "typ_boite_nb_rapp" : "boite_v",
                "conso_urb" : "conso_urb",
                "conso_exurb" : "conso_exurb",
                "conso_mixte" : "conso_mixte",
                "co2" : "co2",
                "co_typ_1" : "co_typ_1",
                "hc" : "hc",
                "nox" : "nox",
                "hcnox" : "hcnox",
                "ptcl" : "particules"
               }

to_drop = []
for c in df_2012.columns:
    if c not in df_2012_cols.keys():
        to_drop.append(c)
        
df_2012 = df_2012.drop(columns=to_drop).rename(columns=df_2012_cols)

## Nettoyage

In [6]:
# suppression des espaces inutiles (par ex. colonne dans la colonne cnit !)
def clean_df_cols(df):
    df_obj = df.select_dtypes(['object'])
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    return df


In [7]:
df_2011 = clean_df_cols(df_2011)
df_2012 = clean_df_cols(df_2012)

## Suppression des doublons du dataset

In [8]:
df_2011.drop_duplicates(subset=['cnit'], inplace=True)
df_2012.drop_duplicates(subset=['cnit'], inplace=True)


## 1er import (raw)

In [9]:
my_dataset = pd.concat([my_dataset, df_2011])

## 2eme import (update)

In [10]:
# chargement des données 2012

# doublons -> update (on conserve 2012)
to_update = my_dataset[my_dataset['cnit'].isin(df_2012['cnit'])]

# suppression des index de my_dataset qui contiennent le même cnit que 2012
my_dataset = my_dataset.drop(index=to_update.index)

# ajout des données de 2012
my_dataset = pd.concat([my_dataset, df_2012])

In [11]:
my_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70078 entries, 0 to 40051
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cnit         70078 non-null  object 
 1   mine         70078 non-null  object 
 2   marque       70078 non-null  object 
 3   modele       70078 non-null  object 
 4   dscom        70078 non-null  object 
 5   carburant    70078 non-null  object 
 6   hybride      39593 non-null  object 
 7   puiss_admin  70078 non-null  float64
 8   puiss_max    70078 non-null  float64
 9   boite_v      70078 non-null  object 
 10  conso_urb    70008 non-null  object 
 11  conso_exurb  70008 non-null  object 
 12  conso_mixte  70011 non-null  object 
 13  masse_v_min  0 non-null      float64
 14  masse_v_max  0 non-null      float64
 15  co2          70011 non-null  float64
 16  co_typ_1     39483 non-null  object 
 17  hc           8077 non-null   object 
 18  nox          39483 non-null  object 
 19  hcno

## export Dataset

In [13]:
my_dataset.to_csv('my_dataset.csv', index=False)
